In [0]:
from pyspark.sql.functions import sum, count, when, col, desc, rank, asc

In [0]:
from pyspark.sql.window import Window

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
# Find race years for which data needs to be processed

race_results_list = spark.read.format('delta').load(f'{presentation_folder_path}/race_results')\
    .filter(f"file_date = '{v_file_date}'")

In [0]:
race_year_list = df_column_to_list(race_results_list, 'race_year')

In [0]:
race_results = spark.read.format('delta').load(f'{presentation_folder_path}/race_results') \
    .filter(col('race_year').isin(race_year_list))

In [0]:
driver_standings_df = race_results\
    .groupBy('race_year', 'driver_name', 'driver_nationality')\
    .agg(sum('points').alias('total_points'),
         count(when(col('position') == 1, True)).alias('wins'))

In [0]:
driver_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))

final_df = driver_standings_df.withColumn('rank', rank().over(driver_rank_spec))

In [0]:
v_merge_condition = 'tgt.driver_name = tmp.driver_name AND tgt.race_year = tmp.race_year'
merge_delta_data(final_df, 'f1_presentation', 'driver_standings', presentation_folder_path, v_merge_condition, 'race_year')